In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from preprocessing import preprocess

In [3]:
data = pd.read_csv("train.csv", index_col=0)
#data_plus = pd.read_csv("training_extra.csv", index_col=0)
#data = pd.concat([data, data_plus], axis=0)

In [4]:
data.tail()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
299995,Adidas,Leather,Small,9.0,No,No,Tote,Blue,12.730812,129.99749
299996,Jansport,Leather,Large,6.0,No,Yes,Tote,Blue,26.633182,19.85819
299997,Puma,Canvas,Large,9.0,Yes,Yes,Backpack,Pink,11.898250,111.41364
299998,Adidas,Nylon,Small,1.0,No,Yes,Tote,Pink,6.175738,115.89080
299999,Under Armour,Canvas,Small,2.0,No,Yes,Backpack,Black,18.568865,26.72762


In [5]:
data = data[:150000]

In [6]:
data = preprocess(data)
data.head()

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Brand_TEmean', 'Brand_TECount',
       'Brand_TESTD', 'Brand_TEVAR', 'Brand_TEnunique', 'Brand_TEmedian',
       'Brand_TEskew', 'Material_TEmean', 'Material_TECount', 'Material_TESTD',
       'Material_TEVAR', 'Material_TEnunique', 'Material_TEmedian',
       'Material_TEskew', 'Size_TEmean', 'Size_TECount', 'Size_TESTD',
       'Size_TEVAR', 'Size_TEnunique', 'Size_TEmedian', 'Size_TEskew',
       'Style_TEmean', 'Style_TECount', 'Style_TESTD', 'Style_TEVAR',
       'Style_TEnunique', 'Style_TEmedian', 'Style_TEskew', 'Color_TEmean',
       'Color_TECount', 'Color_TESTD', 'Color_TEVAR', 'Color_TEnunique',
       'Color_TEmedian', 'Color_TEskew'],
      dtype='object')


,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price,...,Style_TEnunique,Style_TEmedian,Style_TEskew,Color_TEmean,Color_TECount,Color_TESTD,Color_TEVAR,Color_TEnunique,Color_TEmedian,Color_TEskew
0,Jansport,Leather,Medium,7.0,1.0,0.0,Tote,Black,11.611723,112.15875,...,30009,80.47179,0.048537,80.555044,23186,39.242215,1539.951402,17816,79.334665,0.079296
1,Jansport,Canvas,Small,10.0,1.0,1.0,Messenger,Green,27.078537,68.88056,...,30446,81.24117,0.032795,82.351278,23094,39.018596,1522.450851,17857,82.218545,0.012331
2,Under Armour,Leather,Small,2.0,1.0,0.0,Messenger,Red,16.643760,39.17320,...,30446,81.24117,0.032795,80.868385,23635,39.182374,1535.258467,18179,80.456350,0.051953
3,Nike,Nylon,Small,8.0,1.0,0.0,Messenger,Green,12.937220,80.60793,...,30446,81.24117,0.032795,82.351278,23094,39.018596,1522.450851,17857,82.218545,0.012331
4,Adidas,Canvas,Medium,1.0,1.0,1.0,Messenger,Green,17.749338,86.02312,...,30446,81.24117,0.032795,82.351278,23094,39.018596,1522.450851,17857,82.218545,0.012331


In [7]:
data.dtypes

Brand                            object
Material                         object
Size                             object
Compartments                    float64
Laptop Compartment              float64
Waterproof                      float64
Style                            object
Color                            object
Weight Capacity (kg)            float64
Price                           float64
Laptop Compartment_Missing        int32
Waterproof_Missing                int32
Weight Capacity (kg)_Missing      int32
Brand_TEmean                    float64
Brand_TECount                     int64
Brand_TESTD                     float64
Brand_TEVAR                     float64
Brand_TEnunique                   int64
Brand_TEmedian                  float64
Brand_TEskew                    float64
Material_TEmean                 float64
Material_TECount                  int64
Material_TESTD                  float64
Material_TEVAR                  float64
Material_TEnunique                int64


---

FIRST SANDBOX - testing before implementing in preprocessing function

In [8]:
numerical_columns = data.select_dtypes(include='number').columns
categorical_columns = data.select_dtypes(exclude='number').columns

In [9]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Brand_TEmean', 'Brand_TECount',
       'Brand_TESTD', 'Brand_TEVAR', 'Brand_TEnunique', 'Brand_TEmedian',
       'Brand_TEskew', 'Material_TEmean', 'Material_TECount', 'Material_TESTD',
       'Material_TEVAR', 'Material_TEnunique', 'Material_TEmedian',
       'Material_TEskew', 'Size_TEmean', 'Size_TECount', 'Size_TESTD',
       'Size_TEVAR', 'Size_TEnunique', 'Size_TEmedian', 'Size_TEskew',
       'Style_TEmean', 'Style_TECount', 'Style_TESTD', 'Style_TEVAR',
       'Style_TEnunique', 'Style_TEmedian', 'Style_TEskew', 'Color_TEmean',
       'Color_TECount', 'Color_TESTD', 'Color_TEVAR', 'Color_TEnunique',
       'Color_TEmedian', 'Color_TEskew'],
      dtype='object')

In [10]:
categorical_columns

Index(['Brand', 'Material', 'Size', 'Style', 'Color'], dtype='object')

END OF THE FIRST SANDBOX

---

In [11]:
print(f"{(len(numerical_columns) + len(categorical_columns))} = {len(data.columns)}")

48 = 48


In [12]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Brand_TEmean', 'Brand_TECount',
       'Brand_TESTD', 'Brand_TEVAR', 'Brand_TEnunique', 'Brand_TEmedian',
       'Brand_TEskew', 'Material_TEmean', 'Material_TECount', 'Material_TESTD',
       'Material_TEVAR', 'Material_TEnunique', 'Material_TEmedian',
       'Material_TEskew', 'Size_TEmean', 'Size_TECount', 'Size_TESTD',
       'Size_TEVAR', 'Size_TEnunique', 'Size_TEmedian', 'Size_TEskew',
       'Style_TEmean', 'Style_TECount', 'Style_TESTD', 'Style_TEVAR',
       'Style_TEnunique', 'Style_TEmedian', 'Style_TEskew', 'Color_TEmean',
       'Color_TECount', 'Color_TESTD', 'Color_TEVAR', 'Color_TEnunique',
       'Color_TEmedian', 'Color_TEskew'],
      dtype='object')

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer
)
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
import xgboost as xgb

In [14]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Brand_TEmean', 'Brand_TECount',
       'Brand_TESTD', 'Brand_TEVAR', 'Brand_TEnunique', 'Brand_TEmedian',
       'Brand_TEskew', 'Material_TEmean', 'Material_TECount', 'Material_TESTD',
       'Material_TEVAR', 'Material_TEnunique', 'Material_TEmedian',
       'Material_TEskew', 'Size_TEmean', 'Size_TECount', 'Size_TESTD',
       'Size_TEVAR', 'Size_TEnunique', 'Size_TEmedian', 'Size_TEskew',
       'Style_TEmean', 'Style_TECount', 'Style_TESTD', 'Style_TEVAR',
       'Style_TEnunique', 'Style_TEmedian', 'Style_TEskew', 'Color_TEmean',
       'Color_TECount', 'Color_TESTD', 'Color_TEVAR', 'Color_TEnunique',
       'Color_TEmedian', 'Color_TEskew'],
      dtype='object')

In [15]:
ohe_columns = [
    "Brand",
    "Material",
    "Size",
    #"Compartments",
    "Style",
    "Color"
]

columns_to_scale = [
    "Compartments",
    "Weight Capacity (kg)"
] + [col for col in data.columns if "TE" in col]

In [16]:
columns_to_scale

['Compartments',
 'Weight Capacity (kg)',
 'Brand_TEmean',
 'Brand_TECount',
 'Brand_TESTD',
 'Brand_TEVAR',
 'Brand_TEnunique',
 'Brand_TEmedian',
 'Brand_TEskew',
 'Material_TEmean',
 'Material_TECount',
 'Material_TESTD',
 'Material_TEVAR',
 'Material_TEnunique',
 'Material_TEmedian',
 'Material_TEskew',
 'Size_TEmean',
 'Size_TECount',
 'Size_TESTD',
 'Size_TEVAR',
 'Size_TEnunique',
 'Size_TEmedian',
 'Size_TEskew',
 'Style_TEmean',
 'Style_TECount',
 'Style_TESTD',
 'Style_TEVAR',
 'Style_TEnunique',
 'Style_TEmedian',
 'Style_TEskew',
 'Color_TEmean',
 'Color_TECount',
 'Color_TESTD',
 'Color_TEVAR',
 'Color_TEnunique',
 'Color_TEmedian',
 'Color_TEskew']

In [17]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
std = StandardScaler()

In [18]:
#submission_on = False
#preprocess_transformer = FunctionTransformer(
#    preprocess, kw_args={'submission': submission_on}
#)

In [19]:
column_transformer = ColumnTransformer(
    transformers=[
        ("OneHotEncoding", ohe, ohe_columns),
        ("Standardization", std, columns_to_scale)
    ],
    remainder="passthrough",
    n_jobs=-1,
    verbose=True,
    verbose_feature_names_out=False
).set_output(transform="pandas")

In [20]:
pipeline = Pipeline(
    [
        #("Preprocessing", preprocess_transformer),
        ("ColumnTransformer", column_transformer),
        #("Model", xgb.XGBRegressor())
    ]
)

In [21]:
X = data.drop(columns='Price')
y = data['Price']

In [22]:
X = pipeline.fit_transform(X)

In [33]:
float_cols = X.select_dtypes(include='float').columns
X[float_cols] = X[float_cols].astype("float16")

In [34]:
X.head(3)

,Brand_Adidas,Brand_Inconnu,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Inconnu,Material_Leather,Material_Nylon,...,Color_TESTD,Color_TEVAR,Color_TEnunique,Color_TEmedian,Color_TEskew,Laptop Compartment,Waterproof,Laptop Compartment_Missing,Waterproof_Missing,Weight Capacity (kg)_Missing
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.111328,1.113281,-0.085205,-1.281250,1.448242,1.0,0.0,0,0,0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.139282,0.136719,-0.069214,1.156250,-0.899414,1.0,1.0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.851074,0.851562,0.056335,-0.333496,0.489746,1.0,0.0,0,0,0


---
SECOND SANDBOX

In [35]:
from combinations_in_trio import finding_best_interactions_by_combinations_of_three

finding_best_interactions_by_combinations_of_three(X, y)

[('Brand_Adidas', 'Material_Canvas', 'Size_Inconnu'), ('Brand_Adidas', 'Material_Canvas', 'Size_Large'), ('Brand_Adidas', 'Material_Canvas', 'Size_Medium'), ('Brand_Adidas', 'Material_Canvas', 'Size_Small'), ('Brand_Adidas', 'Material_Canvas', 'Style_Backpack'), ('Brand_Adidas', 'Material_Canvas', 'Style_Inconnu'), ('Brand_Adidas', 'Material_Canvas', 'Style_Messenger'), ('Brand_Adidas', 'Material_Canvas', 'Style_Tote'), ('Brand_Adidas', 'Material_Canvas', 'Color_Black'), ('Brand_Adidas', 'Material_Canvas', 'Color_Blue'), ('Brand_Adidas', 'Material_Canvas', 'Color_Gray'), ('Brand_Adidas', 'Material_Canvas', 'Color_Green'), ('Brand_Adidas', 'Material_Canvas', 'Color_Inconnu'), ('Brand_Adidas', 'Material_Canvas', 'Color_Pink'), ('Brand_Adidas', 'Material_Canvas', 'Color_Red'), ('Brand_Adidas', 'Material_Canvas', 'Compartments'), ('Brand_Adidas', 'Material_Canvas', 'Weight Capacity (kg)'), ('Brand_Adidas', 'Material_Canvas', 'Size_TEmean'), ('Brand_Adidas', 'Material_Canvas', 'Size_TECount

{'Brand_Adidas*Material_Canvas*Size_Inconnu': -39.522510703254326,
 'no_column_added': -39.55263107932525}

END OF THE SECOND SANDBOX

---

In [ ]:
# need to find out again the best combinations

import json

with open('best_combinations_22_02.json', 'r') as json_file:
    relevant_combinations = json.load(json_file)

combinations_to_add = list(relevant_combinations.keys())[:50]

for combination in combinations_to_add:
    separation = combination.split("*")
    first_col = separation[0]
    second_col = separation[1]
    X[combination] = X[first_col] * X[second_col]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2796\3740525352.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[combination] = X[first_col] * X[second_col]
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2796\3740525352.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[combination] = X[first_col] * X[second_col]
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2796\3740525352.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance. 

In [ ]:
import json

with open('best_combinations_in_trio_25_02.json', 'r') as json_file:
    relevant_combinations = json.load(json_file)

combinations_to_add = list(relevant_combinations.keys())[:50]

for combination in combinations_to_add:
    separation = combination.split("*")
    first_col = separation[0]
    second_col = separation[1]
    third_col = separation[2]
    X[combination] = X[first_col] * X[second_col] * X[third_col]

In [ ]:
X.head()

,Brand_Adidas,Brand_Inconnu,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Inconnu,Material_Leather,Material_Nylon,...,Material_Nylon*Color_TEmean,Material_TEnunique*Color_TECount,Size_Inconnu*Style_TECount,Material_TECount*Laptop Compartment_Missing,Color_TEnunique*Weight Capacity (kg)_Missing,Color_Red*Style_TEmean,Color_Red*Style_TEmedian,Color_Pink*Weight Capacity (kg)_Missing,Style_Inconnu*Color_TEnunique,Color_Red*Style_TEskew
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.000000,-0.012229,0.0,0.0,-0.0,-0.000000,-0.000000,0.0,-0.0,0.000000
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.000000,0.020078,0.0,-0.0,-0.0,0.000000,0.000000,0.0,-0.0,-0.000000
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,-0.000000,0.002787,0.0,0.0,0.0,0.864883,1.262177,0.0,0.0,-0.601446
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.455711,-0.004165,0.0,-0.0,-0.0,0.000000,0.000000,0.0,-0.0,-0.000000
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.000000,0.020078,0.0,-0.0,-0.0,0.000000,0.000000,0.0,-0.0,-0.000000


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor as KNN

KF = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
model = xgb.XGBRegressor()

---

CROSS VAL

In [ ]:
cv_results = cross_validate(model, X, y, cv=KF, verbose=3, return_train_score=True, scoring='neg_root_mean_squared_error', n_jobs=-1)
print(cv_results['train_score'].mean())
print(cv_results['test_score'].mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  4.4min remaining:  6.7min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.4min finished


-34.86031365291687
-42.70770694315022


In [ ]:
# only with the first twenty : -38.880299988987474

In [ ]:
model = xgb.XGBRegressor()
model.fit(X, y)

KNeighborsRegressor()

---

GRID SEARCH

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
xgboost_param_grid = {
    'n_estimators': [100, 300],  
    'max_depth': [6, 10],  
    'learning_rate': [0.01, 0.2],
    'subsample': [0.85],
    'colsample_bytree': [0.6, 0.8],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [0.1, 1.0],
    'eval_metric': ['logloss'],
    'booster': ['gbtree'],
    'min_child_weight': [1, 3]
}

xgboost_grid = GridSearchCV(
    estimator=model,
    param_grid=xgboost_param_grid,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=True,
    n_jobs=-1
)

In [ ]:
#xgboost_grid.fit(X, y)
#print(f"Best XGBoost Params: {xgboost_grid.best_params_}")
#print(f"Best model's score : {xgboost_grid.best_score_}")
#xgboost_best_model = xgboost_grid.best_estimator_

---

DATA TEST

In [ ]:
test = pd.read_csv('test.csv', index_col=0)
test.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
id,,,,,,,,,
300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953


In [ ]:
test = preprocess(test, submission=True)
test.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Laptop Compartment_Missing,...,Style_TEnunique,Style_TEmedian,Style_TEskew,Color_TEmean,Color_TECount,Color_TESTD,Color_TEVAR,Color_TEnunique,Color_TEmedian,Color_TEskew
id,,,,,,,,,,,,,,,,,,,,,
300000,Puma,Leather,Small,2.0,0.0,0.0,Tote,Green,20.671147,0,...,30009.0,80.47179,0.048537,82.351278,23094.0,39.018596,1522.450851,17857.0,82.218545,0.012331
300001,Nike,Canvas,Medium,7.0,0.0,1.0,Backpack,Green,13.564105,0,...,29648.0,80.50840,0.035718,82.351278,23094.0,39.018596,1522.450851,17857.0,82.218545,0.012331
300002,Adidas,Canvas,Large,9.0,0.0,1.0,Messenger,Blue,11.809799,0,...,30446.0,81.24117,0.032795,81.953437,24090.0,38.613940,1491.036379,18564.0,82.446105,-0.004556
300003,Adidas,Nylon,Large,1.0,1.0,0.0,Messenger,Green,18.477036,0,...,30446.0,81.24117,0.032795,82.351278,23094.0,39.018596,1522.450851,17857.0,82.218545,0.012331
300004,Inconnu,Nylon,Large,2.0,1.0,1.0,Tote,Black,9.907953,0,...,30009.0,80.47179,0.048537,80.555044,23186.0,39.242215,1539.951402,17816.0,79.334665,0.079296


In [ ]:
test = pipeline.transform(test)
test.head()

,Brand_Adidas,Brand_Inconnu,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Inconnu,Material_Leather,Material_Nylon,...,Color_TESTD,Color_TEVAR,Color_TEnunique,Color_TEmedian,Color_TEskew,Laptop Compartment,Waterproof,Laptop Compartment_Missing,Waterproof_Missing,Weight Capacity (kg)_Missing
id,,,,,,,,,,,,,,,,,,,,,
300000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.139340,0.136671,-0.069237,1.156496,-0.899340,0.0,0.0,0,0,0
300001,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.139340,0.136671,-0.069237,1.156496,-0.899340,0.0,1.0,0,0,0
300002,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1.619262,-1.617131,0.206488,1.348890,-1.491413,0.0,1.0,0,0,0
300003,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.139340,0.136671,-0.069237,1.156496,-0.899340,1.0,0.0,0,0,0
300004,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.111167,1.113688,-0.085227,-1.281725,1.448434,1.0,1.0,0,0,0


In [ ]:
for combination in combinations_to_add:
    separation = combination.split("*")
    first_col = separation[0]
    second_col = separation[1]
    test[combination] = test[first_col] * test[second_col]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2796\3679253556.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[combination] = test[first_col] * test[second_col]
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2796\3679253556.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test[combination] = test[first_col] * test[second_col]
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2796\3679253556.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

In [ ]:
preds = pd.concat([pd.DataFrame(test.index), pd.DataFrame(model.predict(test))], axis=1, ignore_index=True)
preds.columns = ['id', 'Price']
print(len(preds))
preds.head()

200000


,id,Price
0,300000,81.095912
1,300001,68.371586
2,300002,74.429320
3,300003,97.668628
4,300004,98.189634


In [ ]:
preds.to_csv('submission.csv', index=False)